## Transition Lopsidedness
For spaced items (lag >= 4) in mixed lists, we determine the proportion of times, given that a subject made a transition from an item in $S_i = {i-2, i-1}$ or $S_j = {j-2, j-1}$ during free recall, that the they would then transition to recall the item in ${i, j}$ where $i$ and $j$ identify the serial positions of the item's two presentations.

To control for the proportion of these transitions expected in the absence of repeated items, we assign the same serial positions used in the mixed lists to corresponding items in the control lists. For each subject, we assign these positions to 100 random shuffles of the control lists, and take the mean across the reshuffled datasets to get a baseline expectation of transition lopsidedness.

In [3]:


def indices_of_repeated_items(presentation_sequence, lag_threshold=4):

    values, counts = np.unique(presentation_sequence, return_counts=True)
    repeated_items = {v: np.where(presentation_sequence == v)[0] for v in values if counts[v] > 1}

    return {key:repeated_items[key] for key in repeated_items if repeated_items[key][1] - repeated_items[key][0] >= lag_threshold}

In [ ]:
mixed_presentations = presentations[list_types==4]
mixed_recalls = trials[list_types==4]

total_si_transitions = 0
total_sj_transitions = 0
total = 0

for trial_index in range(len(mixed_presentations)):
    presentation = mixed_presentations[trial_index]

    # identify identites and indices of all repeated items in presented mixed lists with lag >= 4; our i and j
    i_and_j = indices_of_repeated_items(presentation)

    for repeated_item in i_and_j:

        # define s_i and s_j based on relative positions from i and j
        s_i = presentation[i_and_j[repeated_item][0]-2:i_and_j[repeated_item][0]]
        s_j = presentation[i_and_j[repeated_item][1]-2:i_and_j[repeated_item][1]]

        # in each corresponding recall sequence, identify transitions from S_i or S_j
        # note: this implementation should actually bug out if s_i or s_j 
        s_i_transitions = mixed_recalls[trial_index][np.nonzero(np.logical_or(
            mixed_recalls[trial_index]==s_i[0], mixed_recalls[trial_index]==s_i[1]))[0]+1]

        s_j_transitions =  mixed_recalls[trial_index][np.nonzero(np.logical_or(
            mixed_recalls[trial_index]==s_j[0], mixed_recalls[trial_index]==s_j[1]))[0]+1]

        # and record if i or j is the transitioned-to item or not
        total_si_transitions += np.any(np.isin(s_i_transitions, repeated_item))
        total_sj_transitions += np.any(np.isin(s_j_transitions, repeated_item))
        total += np.logical_or(np.any(np.isin(s_i_transitions, repeated_item)), np.any(np.isin(s_j_transitions, repeated_item)))

In [ ]:
control_presentations = presentations[list_types==1]
control_recalls = trials[list_types==4]

total_control_si_transitions = 0
total_control_sj_transitions = 0
total_control = 0

order = np.arange(len(control_presentations))

# for each shuffling of indices of their control trials
for i in range(100):
    np.random.shuffle(order)

    # map each control trial to a unique mixed trial the subject participated in, and identify the corresponding i and j
    for trial_index in range(len(mixed_presentations)):
        mixed_presentation = mixed_presentations[trial_index]

        # identify identites and indices of all repeated items in presented mixed lists with lag >= 4; our i and j
        i_and_j = indices_of_repeated_items(mixed_presentation)

        for repeated_item in i_and_j:

            # define s_i and s_j based on relative positions from i and j
            s_i = np.array([i_and_j[repeated_item][0]-2, i_and_j[repeated_item][0]-1])
            s_j = np.array([i_and_j[repeated_item][1]-2, i_and_j[repeated_item][1]-1])

            # this time test items are just the indices we track in i_and_j
            test_items = i_and_j[repeated_item]

            # in each corresponding recall sequence, identify transitions from S_i or S_j
            # note: this implementation should actually bug out if s_i or s_j 
            s_i_transitions = control_recalls[order[trial_index]][np.nonzero(np.logical_or(
                control_recalls[order[trial_index]]==s_i[0], control_recalls[order[trial_index]]==s_i[1]))[0]+1]

            s_j_transitions =  control_recalls[order[trial_index]][np.nonzero(np.logical_or(
                control_recalls[order[trial_index]]==s_j[0], control_recalls[order[trial_index]]==s_j[1]))[0]+1]

            # and record if i or j is the transitioned-to item or not
            if not np.any(np.isin(s_j_transitions, test_items)):
                total_control_si_transitions += np.any(np.isin(s_i_transitions, test_items))
                total_control += 1
            elif not np.any(np.isin(s_i_transitions, test_items)):
                total_control_sj_transitions += np.any(np.isin(s_j_transitions, test_items))
                total_control += 1

In [ ]:
total_si_transitions / total_sj_transitions 

4.67741935483871

In [ ]:
total_control_si_transitions/total_control_sj_transitions

1.2512820512820513

## Transition Lopsidedness: Forward Variant
We'll do the same analysis as above, but instead of considering transitions from items in $S_i = \{i-2, i-1\}$ or $S_j = \{j-2, j-1\}$ to items in $\{i, j\}$, we'll measure the rate of transitions from items in $\{i, j\}$ to items in $S_i = \{i+1, i+2\}$ or $S_j = \{j+1, j+2\}$. This is to suppose that the differential encoding mechanism affects forward transitions from differentially encoded item presentations, rather than that it affects the likelihood of transitioning recalls based on those presentations.

Given recall of an item with repeated presentations, what's the relative probability of recalling an item near its first presentation compared to recalling an item near its second presentation?

In [ ]:
mixed_presentations = presentations[list_types==4]
mixed_recalls = trials[list_types==4]

total_si_transitions = 0
total_sj_transitions = 0
total = 0

for trial_index in range(len(mixed_presentations)):
    presentation = mixed_presentations[trial_index]

    # identify identites and indices of all repeated items in presented mixed lists with lag >= 4; our i and j
    i_and_j = indices_of_repeated_items(presentation)

    for repeated_item in i_and_j:

        # define s_i and s_j based on relative positions from i and j
        s_i = presentation[i_and_j[repeated_item][0]-2:i_and_j[repeated_item][0]]
        s_j = presentation[i_and_j[repeated_item][1]-2:i_and_j[repeated_item][1]]

        # in each corresponding recall sequence, identify transitions from repeated item
        repeated_item_transition = mixed_recalls[trial_index][np.nonzero(
            mixed_recalls[trial_index]==repeated_item)[0]+1]

        # and record if transitioned-to item is in Si or Sj is the transitioned-to item or not
        total_si_transitions += np.any(np.isin(s_i, repeated_item_transition))
        total_sj_transitions += np.any(np.isin(s_j, repeated_item_transition))
        total += np.logical_or(np.any(np.isin(s_i, repeated_item_transition)), np.any(np.isin(s_j, repeated_item_transition)))

In [ ]:
control_presentations = presentations[list_types==1]
control_recalls = trials[list_types==4]

total_control_si_transitions = 0
total_control_sj_transitions = 0
total_control = 0

order = np.arange(len(control_presentations))

# for each shuffling of indices of their control trials
for i in range(100):
    np.random.shuffle(order)

    # map each control trial to a unique mixed trial the subject participated in, and identify the corresponding i and j
    for trial_index in range(len(mixed_presentations)):
        mixed_presentation = mixed_presentations[trial_index]

        # identify identites and indices of all repeated items in presented mixed lists with lag >= 4; our i and j
        i_and_j = indices_of_repeated_items(mixed_presentation)

        for repeated_item in i_and_j:

            # define s_i and s_j based on relative positions from i and j
            s_i = np.array([i_and_j[repeated_item][0]-2, i_and_j[repeated_item][0]-1])
            s_j = np.array([i_and_j[repeated_item][1]-2, i_and_j[repeated_item][1]-1])

            # in each corresponding recall sequence, identify transitions from test items mapped from repeated items
            test_items = i_and_j[repeated_item]
            test_item_transition = control_recalls[order[trial_index]][np.nonzero(
                np.logical_or(control_recalls[order[trial_index]]==test_items[0], control_recalls[order[trial_index]]==test_items[1]))[0]+1]

            # and record if s_i or s_j is the transitioned-to item or not, given that 
            if len(test_item_transition) > 0:
                if not np.any(np.isin(s_j, test_item_transition)):
                    total_control_si_transitions += np.any(np.isin(s_i, test_item_transition))
                    total_control += 1
                elif not np.any(np.isin(s_i, test_item_transition)):
                    total_control_sj_transitions += np.any(np.isin(s_j, test_item_transition))
                    total_control += 1